# 第十一课 加密与解密

- [ ] 对称加密与解密
- [ ] 将加密解密操作搭建到Gradio Demo中
- [ ] 利用哈希函数进行口令认证
- [ ] 多模态大模型的Gradio demo

我们发现很多同学会对密码学的过程非常感兴趣。所以在这节课，我们来学习一下密码学中比较重要和基础的一些概念。

我们假设同学Alice和同学Bob正在用一种通常的通讯方式进行文本聊天，比如说QQ或者微信。但是他们不希望其他人在查看聊天记录的时候，能够知道他们聊了什么东西。那有什么办法，可以对聊天的内容进行加密呢？

我们可以假设文本text可以用一串密码key来进行加密，并且这个key只有Alice和Bob知道

那Bob就可以针对key来进行解密

现在我们让ChatGPT来帮忙实现对应锁需要的程序。

---

请帮我实现一对加密和解密的程序

加密程序以文本text和key两个字符串输入，输出加密后的字符串

解密程序以加密后的字符串和key两个字符串输入，输出解密后的字符串

---

这里比较出乎我意料的是，ChatGLM给我输出了最简单的“移位密码”的实现

In [1]:
def encrypt(text, key):
    encrypted_text = ""
    for i in range(len(text)):
        char = text[i]
        if char.isalpha():
            shift = ord(key[i % len(key)].lower()) % 26
            if char.islower():
                encrypted_text += chr((ord(char) - ord('a') + shift) % 26 + ord('a'))
            else:
                encrypted_text += chr((ord(char) - ord('A') + shift) % 26 + ord('A'))
        else:
            encrypted_text += char
    return encrypted_text

def decrypt(encrypted_text, key):
    decrypted_text = ""
    for i in range(len(encrypted_text)):
        char = encrypted_text[i]
        if char.isalpha():
            shift = ord(key[i % len(key)].lower()) % 26
            if char.islower():
                decrypted_text += chr((ord(char) - ord('a') - shift) % 26 + ord('a'))
            else:
                decrypted_text += chr((ord(char) - ord('A') - shift) % 26 + ord('A'))
        else:
            decrypted_text += char
    return decrypted_text

# 示例使用
text = "Hello, World!"
key = "secret"

encrypted = encrypt(text, key)
print("Encrypted:", encrypted)

decrypted = decrypt(encrypted, key)
print("Decrypted:", decrypted)


Encrypted: Sbgvl, Tjbip!
Decrypted: Hello, World!


但是显然，这种密码太容易被暴力破解了

```
移位密码太容易被破解了，有没有更好的加密算法？
```

In [4]:
from cryptography.fernet import Fernet

def encrypt(text, key):
    # 将base64编码的字符串转换为Fernet对象
    fernet = Fernet(key)
    # 加密文本
    encrypted_text = fernet.encrypt(text.encode())
    return encrypted_text

def decrypt(encrypted_text, key):
    # 将base64编码的字符串转换为Fernet对象
    fernet = Fernet(key)
    # 解密文本
    decrypted_text = fernet.decrypt(encrypted_text).decode()
    return decrypted_text

# 生成一个base64编码的密钥
key = Fernet.generate_key()

# 示例使用
text = "Hello, World!"

encrypted = encrypt(text, key)
print("Encrypted:", encrypted)

decrypted = decrypt(encrypted, key)
print("Decrypted:", decrypted)


Encrypted: b'gAAAAABmvaY31EM9fKgm4qy_LE-xRIc8K2PpeE-_cPz-HUFjwdGdQIqe3TpHI-twhXcv7JdKNMp6zK4H4eBp-JAtp9mvo6NonQ=='
Decrypted: Hello, World!


我们知道AES是一个更好的加密

但是AES本身必须要一个特定格式的key

所以我们可以进一步修改这个需求

请帮我实现一对AES加密和解密的程序

加密程序以文本text和key两个字符串输入，输出加密后的字符串

解密程序以加密后的字符串和key两个字符串输入，输出解密后的字符串

如果key本身不符合AES的要求，使用一个确定性的函数把key转化成合法的key

In [5]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import hashlib
import base64

def generate_aes_key(key_str):
    """Generate a valid AES key from a given string."""
    # Use SHA-256 to hash the key_str to ensure it's 32 bytes long
    return hashlib.sha256(key_str.encode()).digest()

def encrypt(text, key_str):
    """Encrypt the given text using AES encryption."""
    key = generate_aes_key(key_str)
    iv = b'0123456789abcdef'  # AES block size is 16 bytes, using a fixed IV for simplicity
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Pad the text to be multiple of AES block size
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_text = padder.update(text.encode()) + padder.finalize()

    encrypted_text = encryptor.update(padded_text) + encryptor.finalize()
    return base64.b64encode(iv + encrypted_text).decode()

def decrypt(encrypted_text, key_str):
    """Decrypt the given encrypted text using AES decryption."""
    key = generate_aes_key(key_str)
    encrypted_text = base64.b64decode(encrypted_text)
    iv = encrypted_text[:16]  # Extract the IV
    encrypted_text = encrypted_text[16:]  # Extract the ciphertext

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_padded_text = decryptor.update(encrypted_text) + decryptor.finalize()

    # Remove padding
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    decrypted_text = unpadder.update(decrypted_padded_text) + unpadder.finalize()
    return decrypted_text.decode()

# 示例用法
text = "Hello, AES!"
key = "my_secret_key"

encrypted_text = encrypt(text, key)
print("Encrypted:", encrypted_text)

decrypted_text = decrypt(encrypted_text, key)
print("Decrypted:", decrypted_text)


Encrypted: MDEyMzQ1Njc4OWFiY2RlZo8XW/t4jUfGmDb2HMYvCR4=
Decrypted: Hello, AES!


可以看到，现在程序达到了我们预期的目标

我们现在开始把这两个函数应用到gradio前端页面

```
{如果不是连续对话，就把上面的代码贴在这里}

我已经实现了encrypt和decrypt函数

我希望实现一个gradio demo来进行方便的操作

这个demo包括一个key的文本输入框

一个未加密文本的大的文本输入框（textArea）

一个加密后的文本的大的文本输入框（textArea）

一个解密后的文本的大的文本输入框（textArea）

一个加密按钮

一个解密按钮

一个清空按钮

我希望加密按钮点击后，把未加密文本输入框的内容加密，并显示在加密后的文本输入框

解密按钮点击后，把加密后的文本输入框的内容解密，并显示在解密后的文本输入框

清空按钮点击后，把所有文本输入框的内容清空

请帮我实现这个gradio demo
```

In [6]:
import gradio as gr
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import hashlib
import base64

def generate_aes_key(key_str):
    """Generate a valid AES key from a given string."""
    return hashlib.sha256(key_str.encode()).digest()

def encrypt(text, key_str):
    """Encrypt the given text using AES encryption."""
    key = generate_aes_key(key_str)
    iv = b'0123456789abcdef'  # AES block size is 16 bytes, using a fixed IV for simplicity
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Pad the text to be multiple of AES block size
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_text = padder.update(text.encode()) + padder.finalize()

    encrypted_text = encryptor.update(padded_text) + encryptor.finalize()
    return base64.b64encode(iv + encrypted_text).decode()

def decrypt(encrypted_text, key_str):
    """Decrypt the given encrypted text using AES decryption."""
    key = generate_aes_key(key_str)
    encrypted_text = base64.b64decode(encrypted_text)
    iv = encrypted_text[:16]  # Extract the IV
    encrypted_text = encrypted_text[16:]  # Extract the ciphertext

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_padded_text = decryptor.update(encrypted_text) + decryptor.finalize()

    # Remove padding
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    decrypted_text = unpadder.update(decrypted_padded_text) + unpadder.finalize()
    return decrypted_text.decode()

def encrypt_text(key, plaintext):
    """Encrypt the plaintext with the given key and return the ciphertext."""
    return encrypt(plaintext, key)

def decrypt_text(key, encrypted_text):
    """Decrypt the ciphertext with the given key and return the plaintext."""
    return decrypt(encrypted_text, key)

def clear_texts():
    """Return empty strings for all text areas."""
    return "", "", "", ""

with gr.Blocks() as demo:
    gr.Markdown("# AES Encryption and Decryption Demo")

    with gr.Row():
        with gr.Column():
            key_input = gr.Textbox(label="Key", placeholder="Enter encryption key", type="text")
            plaintext_input = gr.Textbox(label="Plaintext", placeholder="Enter text to encrypt", type="text", lines=10)
            encrypted_input = gr.Textbox(label="Encrypted Text", placeholder="Encrypted text will appear here", type="text", lines=10)
            decrypted_input = gr.Textbox(label="Decrypted Text", placeholder="Decrypted text will appear here", type="text", lines=10)

        with gr.Column():
            encrypt_button = gr.Button("Encrypt")
            decrypt_button = gr.Button("Decrypt")
            clear_button = gr.Button("Clear")

    encrypt_button.click(fn=encrypt_text, inputs=[key_input, plaintext_input], outputs=encrypted_input)
    decrypt_button.click(fn=decrypt_text, inputs=[key_input, encrypted_input], outputs=decrypted_input)
    clear_button.click(fn=clear_texts, inputs=[], outputs=[key_input, plaintext_input, encrypted_input, decrypted_input])

demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


尝试gradio输出的本地链接，试验一下加密解密这个demo吧

---

## 口令认证

当然，在很多时候我们会遇到一个问题，如果你加密的过程/代码能够被别人看到，那么别人也可以通过这个密码来对你输出的字段进行解密。

很典型的情况，是当我们运行一个在线demo的时候，

如果我们想有一个口令 password 的输入

接着我们要实现一个判断口令是否正确的函数

```python
def check_password(password):
    """Check if the password is correct."""
    return password == "correct_password"
```

如果我们使用这样的方法来判断password，那当一个其他人能够访问到程序代码的时候

他就可以知道口令实际上就是"correct_password"

那我们有没有什么办法让即使是能阅读到代码的人，也无法直接知道密码呢？

实际上这样的算法是存在的，这样的算法就是利用一个相对复杂的哈希函数

假设我们的判断函数是这样实现的


```python
def check_password(password):
    """Check if the password is correct."""
    return hash( password ) == "verify_code"
```

并且hash函数的逆运算并不是一个很容易计算的（实际上要天文时间）。这样的hash函数是存在的，

在密码学上称为单向哈希函数，实际上前面的SHA-256就是一个非常通用的单项函数

我们来让ChatGPT帮我们实现这个函数和配套的函数

---

我希望实现一套非对称加密的函数


```python
def check_password(password):
    """Check if the password is correct."""
    return hash( password ) == "verify_code"
```

使得即使有人能够访问到hash函数的代码和verify_code，也无法反过来计算出password是什么

请帮我实现对应的hash函数

---

In [10]:
import hashlib

def hash(password):
    """Create a SHA-256 hash of the password."""
    # 创建一个sha256对象
    sha_signature = hashlib.sha256(password.encode()).hexdigest()
    return sha_signature

def check_password(password):
    """Check if the password is correct."""
    # 此处需要预设一个正确的hash值，实际应用中这个值应该是安全存储的
    verify_code = "3347e4af6cb1435096396ebcac7ccdee8135adb2d1a11fb0fdb120df3e4e7652"
    return hash(password) == verify_code

# 示例使用
password = "your_secret_password"
print("Hash of password:", hash(password))
print("---")
if check_password(password):
    print("Password is correct.")
else:
    print("Password is incorrect.")


Hash of password: 3347e4af6cb1435096396ebcac7ccdee8135adb2d1a11fb0fdb120df3e4e7652
---
Password is correct.


通过阅读这段代码可以发现

我们可以提前把hash(password)的结果赋值给verify_code

这样即使有人能够访问到hash函数的代码和verify_code，也无法反过来计算出password是什么

只要他访问不到你给password赋值的过程，他是无法知道你的密码。

我们可以接下来写个“欢迎回家”的程序来实践这个过程

```python
import hashlib

def hash(password):
    """Create a SHA-256 hash of the password."""
    # 创建一个sha256对象
    sha_signature = hashlib.sha256(password.encode()).hexdigest()
    return sha_signature

def check_password(password):
    """Check if the password is correct."""
    # 此处需要预设一个正确的hash值，实际应用中这个值应该是安全存储的
    verify_code = "3347e4af6cb1435096396ebcac7ccdee8135adb2d1a11fb0fdb120df3e4e7652"
    return hash(password) == verify_code
```

我希望写一个gradio demo， 如果文本输入是正确的话，显示“欢迎回家”,否则显示 “密码错误”


In [12]:
import gradio as gr
import hashlib

def hash(password):
    """Create a SHA-256 hash of the password."""
    sha_signature = hashlib.sha256(password.encode()).hexdigest()
    return sha_signature

def check_password(password):
    """Check if the password is correct."""
    verify_code = "3347e4af6cb1435096396ebcac7ccdee8135adb2d1a11fb0fdb120df3e4e7652"
    if hash(password) == verify_code:
        return "欢迎回家"
    else:
        return "密码错误"

# 创建Gradio界面
interface = gr.Interface(
    fn=check_password,
    inputs=gr.Textbox(label="请输入密码"),
    outputs="text",
    title="密码验证",
    description="请输入您的密码进行验证。"
)

# 启动界面
interface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


可以实验这个demo，只有当密码正确的时候

才会显示欢迎回家

并且阅读上面的代码，确认这段代码中，并没有对密码进行泄露。

## 多模态大模型的实践

在第9节课的时候我们实验了较多的纯文本模型。随着大型的语言模型发展到了一定的水平，科学家们也开始逐步发展多模态的大型模型。最典型的应用就是可以在对话过程中去插入图片。下面让我们来实践一下

在 https://platform.lingyiwanwu.com/docs#%E8%A7%86%E8%A7%89%E7%90%86%E8%A7%A3 这个页面中，我们可以找到零一万物api的使用说明。

通过阅读文档我们发现

- 对于输入的图像，我们需要把图像转化为base64编码形式的字符串
- 我们要载入api_key，获得授权

解决完这两点，我们就可以实现多模态模型的调用

In [1]:
# 这里因为鲁鲁老师使用了代理所以要运行这一句，同学们不需要运行这一句。
import os
os.environ['HTTP_PROXY'] = 'http://localhost:8234'
os.environ['HTTPS_PROXY'] = 'http://localhost:8234'


这里我们按照我们之前的形式，

接入了我们存在data/01_key.txt的api_key

并且实现了image2base64的函数

In [15]:
from PIL import Image
import base64
from io import BytesIO

def image2base64(image_path):
    # 打开图像
    with Image.open(image_path) as img:
        # 检查图像高度是否超过480
        if img.height > 480:
            # 计算调整后的宽度，以保持宽高比不变
            aspect_ratio = img.width / img.height
            new_height = 480
            new_width = int(new_height * aspect_ratio)
            img = img.resize((new_width, new_height), Image.ANTIALIAS)
        
        # 使用BytesIO在内存中保存调整大小后的图像
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        buffered.seek(0)

        # 将图像转换为Base64编码字符串
        img_base64 = "data:image/jpeg;base64," + base64.b64encode(buffered.read()).decode('utf-8')
    
    return img_base64


from openai import OpenAI
import base64
import httpx

def load_access_token(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()
    
API_BASE = "https://api.lingyiwanwu.com/v1"
API_KEY = load_access_token("data/01_key.txt")
print(API_KEY[:5])

client = OpenAI(
  api_key=API_KEY,
  base_url=API_BASE
)

img_name = "images/car.jpg"

img_base64 = image2base64(img_name)

prompt = "请根据图片内容生成一段详细的文字描述"

completion = client.chat.completions.create(
  model="yi-vision",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt
        },
        {
          "type": "image_url",
          "image_url": {
            "url": img_base64
          }
        }
      ]
    }
  ],
  stream=False
)

9de1e


In [16]:
print(completion.choices[0].message.content)

这张图片中，一辆红色的豪华跑车正在赛道上飞驰。根据其独特的设计和前部的圆形标志，这辆车似乎是一辆宾利 Continental GT。它低矮而宽阔的姿态以及流畅的线条都在强调运动性和速度。车辆的前大灯亮着，表明可能是低光条件或用于强调。轮子正在旋转，捕捉到轮胎和轮毂模糊的瞬间，这突出了汽车的运动状态。汽车后面的背景是一片平坦的景观，远处似乎有一个山丘或山脉，天空阴沉。汽车的车牌号部分可见，写着“DAGI BZV”。赛道本身看起来是由柏油路组成的，边缘有一条白色的线，表示赛道或围栏。


可以看到，这里我们这段代码是可以正常运行的，

现在我们可以要求ChatGPT来重构这段代码，并且实现对应的Gradio Demo。

---

{之前的代码，包括image2base64，load_access_token，client的创建，调用和print的部分}

这段代码可以顺利运行。

我希望

- 重构这段代码，实现一个image2caption函数
- 实现一个gradio demo，上传一张图片，点击“描述”按钮之后，就可以生成详细的描述

---

In [4]:
from PIL import Image
import base64
from io import BytesIO
from openai import OpenAI
import base64
import httpx

def image2base64(image_path):
    with Image.open(image_path) as img:
        if img.height > 480:
            aspect_ratio = img.width / img.height
            new_height = 480
            new_width = int(new_height * aspect_ratio)
            img = img.resize((new_width, new_height), Image.ANTIALIAS)
        
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        buffered.seek(0)
        
        img_base64 = "data:image/jpeg;base64," + base64.b64encode(buffered.read()).decode('utf-8')
    
    return img_base64

def image2caption(image_path, prompt, client):
    img_base64 = image2base64(image_path)
    completion = client.chat.completions.create(
      model="yi-vision",
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt
            },
            {
              "type": "image_url",
              "image_url": {
                "url": img_base64
              }
            }
          ]
        }
      ],
      stream=False
    )
    return completion.choices[0].message.content


In [5]:
import gradio as gr

def load_access_token(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()

API_BASE = "https://api.lingyiwanwu.com/v1"
API_KEY = load_access_token("data/01_key.txt")

client = OpenAI(
  api_key=API_KEY,
  base_url=API_BASE
)

def gradio_image2caption(img):
    prompt = "请根据图片内容生成一段详细的文字描述"
    return image2caption(img, prompt, client)

iface = gr.Interface(
    fn=gradio_image2caption,
    inputs=gr.Image(type="filepath"),
    outputs="text",
    title="图像描述生成器",
    description="上传图片，点击“描述”按钮生成描述。"
)

iface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


![图片描述](images/image_caption.jpg)

从图中可以看到这个demo的运行结果

## 小结

这节课中，我们对密码学中的加密和解密进行了学习，并且实现了一个Gradio的界面进行体验。

同时，我们介绍了哈希函数。如果同学们对密码学有进一步的兴趣，可以学习对称加密和非对称加密。

在后面我们还进一步体验了多模态大模型的应用。